In [17]:
import torch
import math
from transformer_lens import HookedTransformer
from muutils.dictmagic import condense_tensor_dict

In [11]:
def normalize(R):
    with torch.no_grad():
        mean = R.mean(dim=0)
        R = R - mean
        norms = torch.norm(R, p=2, dim=1, keepdim=True)
        R = R/norms
    return R

def cal_cov(R):
    with torch.no_grad():
        Z = torch.nn.functional.normalize(R, dim=1)
        print(Z.shape)
        A = torch.matmul(Z.T, Z)/Z.shape[0]
    return A

def cal_entropy(A):
    with torch.no_grad():
        eig_val = torch.svd(A / torch.trace(A))[1] 
        entropy = - (eig_val * torch.log(eig_val)).nansum().item()
        normalized_entropy = entropy/math.log(A.shape[0])
    return normalized_entropy


In [4]:
prompt = "good job"

In [5]:
MODEL: HookedTransformer = HookedTransformer.from_pretrained("tiny-stories-3M")
#MODEL: HookedTransformer = HookedTransformer.from_pretrained("gpt2")
TOKENIZER = MODEL.tokenizer

input_ids = TOKENIZER.encode(prompt)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loaded pretrained model tiny-stories-3M into HookedTransformer


In [6]:
d_model: int = MODEL.cfg.d_model
n_layers: int = MODEL.cfg.n_layers
n_heads: int = MODEL.cfg.n_heads
d_vocab: int = MODEL.cfg.d_vocab
print(f"{d_model=}, {n_layers=}, {n_heads=}, {d_vocab=}")

d_model=128, n_layers=8, n_heads=16, d_vocab=50257


In [7]:
input_ids

[11274, 1693]

In [13]:
MODEL(torch.tensor(input_ids)).shape

torch.Size([1, 2, 50257])

In [ ]:
MODEL(prompt).shape

torch.Size([1, 3, 50257])

In [14]:
logits, cache = MODEL.run_with_cache(
    "test text",
)

In [18]:
print(condense_tensor_dict(cache, return_format="yaml"))

blocks:
  '[0-7]':
    ln1:
      hook_scale: (1, 3, 1)
      hook_normalized: (1, 3, 128)
    attn:
      '[hook_q, hook_k, hook_v, hook_z]': (1, 3, 16, 8)
      '[hook_attn_scores, hook_pattern]': (1, 16, 3, 3)
    ln2:
      hook_scale: (1, 3, 1)
      hook_normalized: (1, 3, 128)
    mlp:
      '[hook_pre, hook_post]': (1, 3, 512)
    '[hook_resid_pre, hook_attn_out, hook_resid_mid, hook_mlp_out, hook_resid_post]': (1,
      3, 128)
ln_final:
  hook_scale: (1, 3, 1)
  hook_normalized: (1, 3, 128)
'[hook_embed, hook_pos_embed]': (1, 3, 128)



In [20]:
with torch.no_grad():
   R = cache["blocks.0.hook_resid_post"][0]
   R = normalize(R)
   A = cal_cov(R)
   Entropy = cal_entropy(A) 
    
print(Entropy)

torch.Size([3, 128])


/var/folders/08/h4ttblxd6m5fjbzyhmxs4bk00000gn/T/ipykernel_75155/2033687359.py:18: UserWarning: The operator 'aten::linalg_svd' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  eig_val = torch.svd(A / torch.trace(A))[1]


0.13609095266154592
